<a href="https://colab.research.google.com/github/edgar-anco/Grupo-8-Complejidad-Algoritmica/blob/main/TF_ANCO_PEREZ_TRUJILLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Se importan las librerias y se crean las clases nodo y arista
#Esto servira para crear los marcadores y lineas dentro del mapa usando Folium
import csv
import folium
from folium.plugins import FastMarkerCluster
from geopy.distance import geodesic
from itertools import combinations

archivo_csv = 'dataset.csv'

class Node:
    def __init__(self, id, latitude, longitude):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude

class Edge:
    def __init__(self, node1, node2, distance):
        self.node1 = node1
        self.node2 = node2
        self.distance = distance


In [ ]:
#Funciones de UFDS, para unir conjuntos
def find_set(sets, node):
    for set in sets:
        if node in set:
            return set
    return None


#Funciones de UFDS, para unir y encontrar conjuntos

def union_sets(sets, set1, set2):
    sets.remove(set1) #remueve un elemento en especifico, set1
    sets.remove(set2)
    sets.append(set1.union(set2)) #une el conjunto 1 y el 2 sin repetir elementos, y crea un nuevo conju

In [ ]:
#Se añaden los nodos y aristas a los arreglos correspondientes

# Leer los datos del archivo CSV y los guarda cada nodo
# en un arreglo de nodos
#estos nodos tambien seran almacenados en un conjunto sets[], para ejecutar el UFDS
nodes = []
with open(archivo_csv, 'r') as archivo:
    lector_csv = csv.reader(archivo, delimiter=';')
    next(lector_csv)
    for linea in lector_csv:
        id = linea[0]
        latitude = float(linea[3])
        longitude = float(linea[4])
        node = Node(id, latitude, longitude)
        nodes.append(node)




#Añade todos los caminos posibles entre un nodo y el resto de nodos
#Aqui esta el problema
edges = []

for i in range(len(nodes)):
    for j in range(i + 1, len(nodes)):
        node1 = nodes[i]
        node2 = nodes[j]
        distance = geodesic((node1.latitude, node1.longitude), (node2.latitude, node2.longitude)).kilometers
        edge = Edge(node1, node2, distance)
        edges.append(edge)
#3m, 0.8s-> menos eficiente

#for node1, node2 in combinations(nodes, 2):
#   distance = geodesic((node1.latitude, node1.longitude), (node2.latitude, node2.longitude)).meters
#   edge = Edge(node1, node2, distance)
#   edges.append(edge)
#2m,56s -> mas eficiente

In [ ]:
#Algoritmo de Kruskal

#Se ordena la lista de aristas por distancia, empezando por la de menor peso
edges.sort(key=lambda x: x.distance)

# sets = [
#     {Node1},
#     {Node2},
#     {Node3},
#     {Node4},
#     {Node5}
# ]
sets = []
for node in nodes:
    sets.append({node})


#Se crea un arreglo de conexiones que serian las aristas
#resultantes que forman parte del camino minimo
connections = []
for edge in edges:
    set1 = find_set(sets, edge.node1)
    set2 = find_set(sets, edge.node2)
    if set1 != set2: #verifica que no esten en el mismo conjunto, para no crear bucles
        union_sets(sets, set1, set2)
        connections.append(edge)


In [ ]:
# Se crea el mapa
m = folium.Map(width=800, height=400)

#Se unen los marcadores en grupos usando cluster para que el programa
#vaya con fluidez al navegar por el mapa
fast_marker_cluster = FastMarkerCluster([], name="Cluster").add_to(m)


#Se crean los marcadores, que son la representacion de los nodos en el mapa
for node in nodes:
    fast_marker_cluster.add_child(
        folium.Marker(
            location=[node.latitude, node.longitude],
            popup=f"Name: {node.id}"
        )
    )


#Se crean las conexiones obtenidas al usar kruskal
for connection in connections:
    location1 = (connection.node1.latitude, connection.node1.longitude)
    location2 = (connection.node2.latitude, connection.node2.longitude)
    folium.PolyLine(locations=[location1, location2],
                    tooltip=f"Distance: {connection.distance:.2f} km").add_to(m)

m